# NER und POS

In [1]:
import sqlite3
import re
import pandas as pd

amazon_sql = 'data/amazon/amazon.db'
ner_sql = 'data/data.db' #db -> ner
pos_sql = 'data/wiktionary/wiktionary.db' # db -> lexikon
text_sql = 'data/text_processing/text_processing.db' #db -> spellcheck

In [2]:
# Lade Amazon_raw
conn=sqlite3.connect(amazon_sql)
cur = conn.cursor()
cur.execute('SELECT asin, ID FROM dvd')
getasin={ID:asin for asin, ID in cur}
conn.close() 

#test
getasin[0]

'B002GH1M5W'

In [3]:
# Lade Asin-Match
conn=sqlite3.connect(ner_sql)
cur = conn.cursor()
cur.execute('SELECT asin, ID_p0, ID_p1, ID_p2, ID_p3, ID_p4, ID_p5, ID_p6 FROM titlematch')
matches={asin:(ID_p0, ID_p1, ID_p2, ID_p3, ID_p4, ID_p5, ID_p6) for asin, ID_p0, ID_p1, ID_p2, ID_p3, ID_p4, ID_p5, ID_p6 in cur}
conn.close() 

#test
matches[getasin[0]]

('180033149',
 '180033149',
 '164135885',
 '180033149',
 '180033149',
 '180033149',
 '172970458')

In [4]:
# Lade NER
conn=sqlite3.connect(ner_sql)
cur = conn.cursor()
cur.execute('SELECT key, pos FROM ner')
ner_pos={token:quick_pos for token, quick_pos in cur}
ner_pos_low={token.lower():value for token, value in ner_pos.items()}
conn.close() 

#test
ner_pos['180033149|Wolverine']

'title|figur'

In [5]:
# Lade POS-Tagger
conn=sqlite3.connect(pos_sql)
cur = conn.cursor()
cur.execute('SELECT token, quick_stemm, quick_pos FROM lexikon')
pos_tag={token:(quick_stemm,quick_pos) for token, quick_stemm, quick_pos in cur}
pos_tag_low={token.lower():value for token, value in pos_tag.items()}
conn.close() 

#test
pos_tag['geladen']

('laden', 'Verb')

In [6]:
# Example
conn=sqlite3.connect(text_sql)
cur = conn.cursor()
cur.execute('SELECT ID,text FROM clean')

for ID, text in cur:
    sID=ID
    asin=getasin[ID]
    IDs=list(set(matches[asin]))
    
    tokens=text.split(' ')
    
    tags=[]    
    for tok in tokens:
        stok=tok
        npos='None'
        ppos='None'
        for ID in IDs:
            key=str(ID)+'|'+tok
            try:
                npos=ner_pos[key]
                break;
            except:
                try:
                    npos=ner_pos_low[key.lower()]
                    break;
                except:
                    try:
                        tok=pos[tok][0]
                    except:
                        pass
                    for ID in IDs:
                        key=str(ID)+'|'+tok
                        try:
                            npos=ner_pos[key]
                            break;
                        except:
                            try:
                                npos=ner_pos_low[key.lower()]
                                break;
                            except:
                                npos='None'

                
        try:
            tag=pos_tag[stok][1]
        except:
            try:
                tag=pos_tag[stok.lower().title()][1]
            except:
                try:
                    tag=pos_tag_low[stok.lower()][1]
                except:
                    tag='None' 
                    
        npos=re.sub('[_][0-9]','',npos)
        tags.append((tok,npos,tag))
        
    x_ner=['None' for i in tags]
    skip=0
    for i in range(len(tags)-1):
        if i>=skip:
            tup=tags[i]
            tag=tup[1].split('|')
            isect=tag
            try:
                for j in range(i+1, len(tags)):
                    tup2=tags[j]
                    tag2=tup2[1].split('|')
                    last=isect.copy()
                    isect=[t for t in tag2 if t in last]
                    if len(isect)==0:
                        x_ner[i:j]=[last[0] for x in x_ner[i:j]]
                        skip=j
                        
                break;
            except:
                pass
                
                
    
                    

    #print(sID)
    if sID==0:   
        break;
        
        
conn.close() 

df=pd.DataFrame(tags,columns=['token','ner','pos'])
df['x_ner']=x_ner
pos=list(df['pos'])
guess=x_ner
final=pos
for row in df.iterrows():
    i,line=row
    try:
        if line['pos'] not in ['Substantiv', 'Adjektiv', 'Komparativ','Superlativ'] and x_ner[i-1]!=x_ner[i] and x_ner[i]!=x_ner[i+1]:
            guess[i]=line['pos']
    except:
        pass
    if line['pos']=='None':
        guess[i]=line['x_ner']
    if guess[i][0].islower():
        final[i]=guess[i].replace('title','titel').split('_')[0]
    else:
        try:
            final[i]=pos_tag[line['token']][0].split('|')[0]
        except:
            try:
                final[i]=pos_tag_low[line['token'].lower()][0].split('|')[0]
            except:
                final[i]=line['token']
            
df['guess']=guess
df['final']=final
df[:13]

,token,ner,pos,x_ner,guess,final
0,X,title,Buchstabe,title,title,titel
1,MEN,title|title,None,title,title,titel
2,ORIGINS,title|title,None,title,title,titel
3,WOLVERINE,title|figur,None,title,title,titel
4,präsentiert,None,Verb,None,None,präsentieren
5,uns,None,personalpronomen,None,None,uns
6,die,None,Artikel,None,None,die
7,Vorgeschichte,None,Substantiv,None,None,Vorgeschichte
8,des,title|title,Artikel,title,Artikel,des
9,coolen,None,Adjektiv,None,None,cool


# NER

In [7]:
# Example
conn=sqlite3.connect(text_sql)
cur = conn.cursor()
cur.execute('SELECT ID,text FROM spellcheck')


data=[]

for ID, text in cur:
    sID=ID
    asin=getasin[ID]
    IDs=list(set(matches[asin]))
    
    tokens=text.split(' ')
    
    tags=[]    
    for tok in tokens:
        stok=tok
        npos='None'
        ppos='None'
        for ID in IDs:
            key=str(ID)+'|'+tok
            try:
                npos=ner_pos[key]
                break;
            except:
                try:
                    npos=ner_pos_low[key.lower()]
                    break;
                except:
                    try:
                        tok=pos[tok][0]
                    except:
                        pass
                    for ID in IDs:
                        key=str(ID)+'|'+tok
                        try:
                            npos=ner_pos[key]
                            break;
                        except:
                            try:
                                npos=ner_pos_low[key.lower()]
                                break;
                            except:
                                npos='None'
                
        try:
            tag=pos_tag[stok][1]
        except:
            try:
                tag=pos_tag[stok.lower().title()][1]
            except:
                try:
                    tag=pos_tag_low[stok.lower()][1]
                except:
                    tag='None' 
                    
        npos=re.sub('[_][0-9]','',npos)
        tags.append((tok,npos,tag))
        
    x_ner=['None' for i in tags]
    skip=0
    for i in range(len(tags)-1):
        if i>=skip:
            tup=tags[i]
            tag=tup[1].split('|')
            isect=tag
            try:
                for j in range(i+1, len(tags)):
                    tup2=tags[j]
                    tag2=tup2[1].split('|')
                    last=isect.copy()
                    isect=[t for t in tag2 if t in last]
                    if len(isect)==0:
                        x_ner[i:j]=[last[0] for x in x_ner[i:j]]
                        skip=j
                        
                break;
            except:
                pass
                
                
    
                    

 
        
    df=pd.DataFrame(tags,columns=['token','ner','pos'])
    df['x_ner']=x_ner
    pos=list(df['pos'])
    guess=x_ner
    final=pos
    for row in df.iterrows():
        i,line=row
        try:
            if line['pos'] not in ['Substantiv', 'Adjektiv', 'Komparativ','Superlativ'] and x_ner[i-1]!=x_ner[i] and x_ner[i]!=x_ner[i+1]:
                guess[i]=line['pos']
        except:
            pass
        if line['pos']=='None':
            guess[i]=line['x_ner']
        if guess[i][0].islower():
            final[i]=guess[i].replace('title','titel').split('_')[0]
        else:
            try:
                final[i]=pos_tag[line['token']][0].split('|')[0]
            except:
                try:
                    final[i]=pos_tag_low[line['token'].lower()][0].split('|')[0]
                except:
                    final[i]=line['token']

    #df['guess']=guess
    #df['final']=final

    
    final_collapse=[]
    for tok in final:
        try:
            if tok!= final_collapse[-1]:
                final_collapse.append(tok)
        except:
            final_collapse.append(tok)
            
    data.append((sID,' '.join(final_collapse)))
    
    if sID%5000==0:
        print(str(sID))
conn.close() 
print(str(sID))

0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
91515


In [8]:
df=pd.DataFrame(data, columns=['ID','text'])

db='ner' 
conn = sqlite3.connect(text_sql)
cur = conn.cursor()                                 

wildcards = ','.join(['?'] * len(df.columns))              

cur.execute("drop table if exists %s" % db)

col_str = '"' + '","'.join(df.columns) + '"'
cur.execute("create table %s (%s)" % (db, col_str))

cur.executemany("insert into %s values(%s)" % (db, wildcards), [tuple(x) for x in df.values])

conn.commit()
conn.close()